In [ ]:
from google import genai
from google.genai import types
from pydantic import BaseModel
from typing import Literal
from pathlib import Path

In [3]:
client = genai.Client(
    vertexai=True,
    location='europe-west4'
)

In [4]:
audio_path = Path("../data/customer_support_call.m4a")

audio = types.Part.from_bytes(
    data=audio_path.read_bytes(),
    mime_type="audio/x-m4a"
)

In [27]:
class CallSummary(BaseModel):
    customer_email: str
    topics_discussed: list[str]
    problem_description: str
    root_cause: str
    solution: str
    problem_resolved: bool
    customer_happiness: Literal[
        "Unhappy",
        "Neutral",
        "Happy",
    ]

response = client.models.generate_content(
    model='gemini-2.0-flash-001',
    contents=[
        "Find below a SparkleNet Customer Support conversation.",
        audio,
    ],
    config=types.GenerateContentConfig(
        temperature=0.2,
        response_mime_type='application/json',
        response_schema=CallSummary,
    ),
)   
call_summary: CallSummary = response.parsed
call_summary

CallSummary(customer_email='john.davis@email.com', topics_discussed=['Login issues', 'Password reset', 'Browser troubleshooting', 'Multi-factor authentication (MFA)'], problem_description='Customer is unable to log in to their SparkleNet account despite resetting their password and being sure they are using the correct credentials.', root_cause='The password reset from the previous day was not fully processed due to a temporary glitch in the system.', solution='The customer support agent manually triggered a new password reset. The customer received an email with a link to create a new password and was able to log in after resetting it.', problem_resolved=True, customer_happiness='Happy')

In [28]:
print(call_summary.model_dump_json(indent=2))

{
  "customer_email": "john.davis@email.com",
  "topics_discussed": [
    "Login issues",
    "Password reset",
    "Browser troubleshooting",
    "Multi-factor authentication (MFA)"
  ],
  "problem_description": "Customer is unable to log in to their SparkleNet account despite resetting their password and being sure they are using the correct credentials.",
  "root_cause": "The password reset from the previous day was not fully processed due to a temporary glitch in the system.",
  "solution": "The customer support agent manually triggered a new password reset. The customer received an email with a link to create a new password and was able to log in after resetting it.",
  "problem_resolved": true,
  "customer_happiness": "Happy"
}


## Cost estimates

In [19]:
response.usage_metadata.total_token_count \
/ 1_000_000 \
* 1.00

0.008323